In [1]:
import os
import sys
import numpy as np
from tqdm.notebook import tqdm
import plotly.express as px

In [2]:
BASE_DIR = os.getcwd()

RECTIFIED_IMAGES = os.path.join(BASE_DIR, "images/augmented/")
DISPARITY_OUTPUT = os.path.join(BASE_DIR, "images/disparity_maps/")
CALIBRATION_DATA = os.path.join(BASE_DIR, "data/stereo_calibration_data.npz")

In [3]:
sys.path.append(BASE_DIR) 
from src.disparity import disparity

In [ ]:
# disparity.optimize_sgbm_params_from_dir(RECTIFIED_IMAGES)

Scanning file (names):   0%|          | 0/5920 [00:00<?, ? file/s]

Creating File-Pairs:   0%|          | 0/5920 [00:00<?, ? pair/s]

Total combinations to try: 288


Optimizing SGBM params:   0%|          | 0/288 [00:00<?, ?combo/s]

KeyboardInterrupt: 

In [9]:
dis = disparity.disparity_depth_estimation(RECTIFIED_IMAGES, DISPARITY_OUTPUT, CALIBRATION_DATA)

Scanning file (names):   0%|          | 0/5919 [00:00<?, ? file/s]

Creating File-Pairs:   0%|          | 0/5919 [00:00<?, ? pair/s]

Computing disparity maps:   0%|          | 0/1 [00:00<?, ?pair/s]

Disparity stats:
	min: -1.0
	max: 127.0
	mean: 24.017675


In [ ]:
dis[500][500:504]

AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [6]:
np.max(dis), np.min(dis), np.mean(dis), np.std(dis)

(np.float32(127.0),
 np.float32(-1.0),
 np.float32(24.017675),
 np.float32(41.485844))

In [17]:
disparities_per_label = disparity.compute_disparities_per_label(RECTIFIED_IMAGES, stop=600)

Scanning file (names):   0%|          | 0/5919 [00:00<?, ? file/s]

Creating File-Pairs:   0%|          | 0/5919 [00:00<?, ? pair/s]

Computing disparity maps:   0%|          | 0/600 [00:00<?, ?pair/s]

KeyboardInterrupt: 

In [ ]:
histogram_data = {}
for label, disparity_matrix in tqdm(disparities_per_label,
                                  desc="",
                                  unit=""):
    # Filter out all -1
    np_disparity_matrix = np.array(disparity_matrix)
    filtered_disparity_matrix = np_disparity_matrix[np_disparity_matrix != -1]
    # Cast to integer
    filtered_disparity_matrix = filtered_disparity_matrix.astype(int)
    histogram_data[label] = {
        "min": np.min(filtered_disparity_matrix),
        "max": np.max(filtered_disparity_matrix),
        "mean": np.mean(filtered_disparity_matrix),
        "std": np.std(filtered_disparity_matrix),
        "values": filtered_disparity_matrix
    }

In [ ]:
for label, data in histogram_data.items():
    fig = px.histogram(
        x=data["values"],
        title=f"Disparity Histogram for {label}",
        labels={"x": "Disparity Value", "y": "Count"},
        # nbins=50
    )
    fig.add_annotation(
        text=f"Min: {data['min']:.2f}, Max: {data['max']:.2f}, Mean: {data['mean']:.2f}, Std: {data['std']:.2f}",
        showarrow=False,
        x=0.5,
        y=-0.15,
        font=dict(size=12)
    )
    fig.show()  